In [ ]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import getpass
import csv

In [ ]:
# Set up postgres connection
conn = psycopg2.connect(
    database="mimic", 
    user=$your_username$, 
    password=getpass.getpass("Enter postgres password"), 
    host="127.0.0.1", 
    port="5432",
    options=f'-c search_path=mimiciii')

## 1. Identify all 'heart disease' patients

In [ ]:
# Get all the patient ids with heart disease
"""
lists of ICD 9 codes (related to heart diseases):

393-398  Chronic Rheumatic Heart Disease
410-414  Ischemic Heart Disease
420-429  Other Forms Of Heart Disease
"""

heart_disease_subject_ids = pd.read_sql(
    """
    SELECT DISTINCT(subject_id)
    FROM diagnoses_icd
    WHERE (
        icd9_code LIKE '393%' OR
        icd9_code LIKE '394%' OR
        icd9_code LIKE '395%' OR
        icd9_code LIKE '396%' OR
        icd9_code LIKE '397%' OR
        icd9_code LIKE '398%' OR
        icd9_code LIKE '410%' OR
        icd9_code LIKE '411%' OR
        icd9_code LIKE '412%' OR
        icd9_code LIKE '413%' OR
        icd9_code LIKE '414%' OR
        icd9_code LIKE '420%' OR
        icd9_code LIKE '421%' OR
        icd9_code LIKE '422%' OR
        icd9_code LIKE '423%' OR
        icd9_code LIKE '424%' OR
        icd9_code LIKE '425%' OR
        icd9_code LIKE '426%' OR
        icd9_code LIKE '427%' OR
        icd9_code LIKE '428%' OR
        icd9_code LIKE '429%' 
    );
    """, conn)

In [ ]:
heart_disease_subject_ids.shape

In [ ]:
# Convert to a set for filtering
heart_disease_id_set = set(heart_disease_subject_ids['subject_id'])

## 2. Retrieve all admission ids from last 12 months since each patient's last admission

In [ ]:
# Get the subtraction between all admission times and the last admission by each patient; in year unit
admissions_diff = pd.read_sql(
    """
    SELECT a.subject_id, a.hadm_id,
    ROUND((cast(a.admittime as date)-cast(last_admission_time.max_admittime as date))/365.242,2) AS diff_from_last 
    FROM admissions AS a
    LEFT JOIN
        (SELECT subject_id,  MAX(admittime) AS max_admittime
        FROM admissions
        GROUP BY subject_id
        ) AS last_admission_time
    ON a.subject_id=last_admission_time.subject_id;
    """, conn)

In [ ]:
admissions_diff.head()

In [ ]:
# Get all the admissions from last 12 months since each patient's last admission ('diff_from_last' >= -1 <year>)
admissions_last_year = admissions_diff[admissions_diff['diff_from_last'] >= -1]

In [ ]:
admissions_last_year.head()

In [ ]:
# Convert all hadm_id's into a set
hadm_id_set = set(admissions_last_year['hadm_id'])

## 3. Get all drug events and process them by remove 'stopword' events

In [ ]:
# Get all the drug events from table 'inputevents_mv'
drug_events = pd.read_sql(
    """
    SELECT im.subject_id, im.hadm_id, im.starttime, im.itemid, di.abbreviation
    FROM inputevents_mv as im
    JOIN d_items as di
    ON im.itemid=di.itemid;
    """, conn)

In [ ]:
drug_events.shape

In [ ]:
drug_events.head()

### 3.1 Filter drug events by the admission id set and heart disease patient ids

In [ ]:
# Filter all drug events that are in the admission id set (admissions from the last 12 month of each patient's last admission)
drug_events_last_year = drug_events[drug_events['hadm_id'].isin(hadm_id_set)]

In [ ]:
# Filter by heart disease patient ids
drug_events_filtered = drug_events_last_year[drug_events_last_year['subject_id'].isin(heart_disease_id_set)]

In [ ]:
drug_events_filtered.head()

### 3.2 Drop the duplicated items (which indicates different doses in the same session)

In [ ]:
# # Uncomment to view duplicated 'itemid's from the same session
# drug_events_filtered.groupby(by='subject_id').apply(lambda x: x.sort_values('itemid'))

In [ ]:
# Drop the duplicates (due to different doses) in the same input session
drug_events_filtered2 = drug_events_filtered.drop_duplicates()

In [ ]:
# # Sanity check, uncomment to view that there are no more duplicates from the same input session
# drug_events_filtered2.groupby(by='subject_id').apply(lambda x: x.sort_values('itemid'))

### 3.3 Remove 'stopword' events (too frequent counts or too rare)

In [ ]:
# Count itemid values
itemid_counts = drug_events_filtered2['itemid'].value_counts()

itemid_counts2 = itemid_counts.reset_index()
itemid_counts2 = itemid_counts2.rename(columns={"index": "itemid", "itemid":"counts"})

In [ ]:
itemid_counts2.head()

In [ ]:
# Add 'proportion' of each itemid to the table 
itemid_counts2['proportion'] = itemid_counts2['counts']/sum(itemid_counts2['counts'])

In [ ]:
itemid_counts2.head(10)

In [ ]:
# Remove these proportions that are larger than 4.1% or the count is less than 5
itemid_counts3 = itemid_counts2[(itemid_counts2['proportion'] <= 0.041) & (itemid_counts2['counts'] >= 5)]

In [ ]:
# Convert to a set - which contains item ids that are neither too frequent nor too rare
itemid_set = set(itemid_counts3['itemid'])

In [ ]:
drug_events_filtered3 = drug_events_filtered2[drug_events_filtered2['itemid'].isin(itemid_set)]

In [ ]:
drug_events_filtered3.head()

In [ ]:
drug_events_filtered3.shape

### 3.4 Group drug events by each patient, and sort by session time and then 'itemid'

In [ ]:
# Group by 'subject_id', and sort by 'starttime' and then 'itemid'
drug_events_only = drug_events_filtered3.groupby(by='subject_id').apply(lambda x: x.sort_values('starttime'))['itemid'].reset_index(level=[1], drop=True)

In [ ]:
drug_events_only.head()

In [ ]:
# Convert to a sequnce of drug events for each patient
drug_events_by_patient = drug_events_only.groupby(by='subject_id').apply(list)

drug_events_by_patient2 = drug_events_by_patient.reset_index()

In [ ]:
drug_events_by_patient2.head()

In [ ]:
drug_events_by_patient2.shape

### 3.5 Filter by event length (resulted sequence length is between 3 to 50)

In [ ]:
# Add 'count' of itemid to the table
drug_events_by_patient2['count'] = [len(events) for events in drug_events_by_patient2['itemid']]

In [ ]:
drug_events_by_patient2['count'].describe()

In [ ]:
# Filter by sequence length
drug_events_by_patient3 = drug_events_by_patient2[drug_events_by_patient2['count'].apply(lambda x: True if x >=3 and x <=50 else False)]

In [ ]:
drug_events_by_patient3.shape

## 4. Get all procedure codes

In [ ]:
procedure_codes = pd.read_sql(
    """
    SELECT a.admittime, procedures.* 
    FROM admissions AS a
    RIGHT JOIN
        (SELECT pi.subject_id, pi.hadm_id, pi.seq_num, pi.icd9_code, dip.short_title
        FROM procedures_icd AS pi
        JOIN d_icd_procedures AS dip
        ON pi.icd9_code=dip.icd9_code) AS procedures
    ON a.hadm_id=procedures.hadm_id;
    """, conn)

In [ ]:
procedure_codes.head()

### 4.1 Filter by the admission id set and heart disease patient ids

In [ ]:
# Filter by the admission id set (admissions from the last 12 month of each patient's last admission)
procedure_codes_last_year = procedure_codes[procedure_codes['hadm_id'].isin(hadm_id_set)]

In [ ]:
# Filter by heart disease patient ids
procedure_codes_filtered = procedure_codes_last_year[procedure_codes_last_year['subject_id'].isin(heart_disease_id_set)]

In [ ]:
procedure_codes_filtered.head()

In [ ]:
procedure_codes_filtered.shape

In [ ]:
procedure_codes_filtered2 = procedure_codes_filtered.drop(['short_title', 'hadm_id'], axis=1)

### 4.2 Group by subject_id and sort by admittime and seq_num

In [ ]:
# group by subject_id and sort by admittime (first) and seq_num (second)
procedure_codes_filtered3 = procedure_codes_filtered2.groupby(by='subject_id').apply(lambda x: x.sort_values(['admittime', 'seq_num']))

In [ ]:
procedure_codes_filtered3.head()

In [ ]:
procedure_codes_filtered4 = procedure_codes_filtered3.reset_index(level=[0,1], drop=True)

In [ ]:
procedure_codes_filtered4.head()

In [ ]:
# Group all the sequential events by patients
procedures_by_patient = procedure_codes_filtered4.groupby(by='subject_id', axis=0)['icd9_code'].apply(list)

procedures_by_patient2 = procedures_by_patient.reset_index()

In [ ]:
procedures_by_patient2.head()

### 4.3 Check the sequence length of procedures for each patient

In [ ]:
# Add 'count' of itemid to the table
procedures_by_patient2['count'] = [len(codes) for codes in procedures_by_patient2['icd9_code']]

In [ ]:
procedures_by_patient2['count'].describe()

## 5. Merge drug events and procedures

In [ ]:
# Merge drug events (filtered by event length between 3 and 50) and procedures
drug_events_procedures_merged = pd.merge(drug_events_by_patient3, procedures_by_patient2, how='inner', on='subject_id')

In [ ]:
drug_events_procedures_merged.shape

In [ ]:
drug_events_procedures_merged.head()

In [ ]:
drug_events_procedures_merged['total_count'] = drug_events_procedures_merged['count_x'] + drug_events_procedures_merged['count_y']

In [ ]:
drug_events_procedures_merged['total_count'].describe()

In [ ]:
# Remove NA values
drug_events_procedures_merged2 = drug_events_procedures_merged[drug_events_procedures_merged['icd9_code'].notna()]

# Remove columns of counts
drug_events_procedures_merged3 = drug_events_procedures_merged2.drop(['count_x', 'count_y', 'total_count'], axis=1)

# Rename columns
drug_events_procedures_merged4 = drug_events_procedures_merged3.rename(columns={"icd9_code": "procedure_codes", "itemid":"drug_events"})

In [ ]:
drug_events_procedures_merged4.head()

In [ ]:
drug_events_procedures_merged4.shape

## 6. Add survival flag (1: survival, 0: death)

In [ ]:
#expire_flag: 1 indicates death in the hospital, and 0 indicates survival to hospital discharge.
survival_subject_ids = pd.read_sql(
    """
    SELECT subject_id FROM patients
    WHERE expire_flag=0;
    """, conn)

In [ ]:
# Convert to a set of survival ids
survival_id_set = set(survival_subject_ids['subject_id'])

In [ ]:
drug_events_procedures_merged4['survival'] = [1 if idx in survival_id_set else 0 for idx in drug_events_procedures_merged4['subject_id']]

In [ ]:
drug_events_procedures_merged4.head()

In [ ]:
drug_events_procedures_merged4['survival'].value_counts()

## 7. Export as the input format for DRG framework

In [ ]:
final_merged = drug_events_procedures_merged4.copy()

In [ ]:
# Split into positive/negative data
neg_data = final_merged[final_merged['survival'] == 0].drop(columns=['subject_id', 'survival'])
pos_data = final_merged[final_merged['survival'] == 1].drop(columns=['subject_id', 'survival'])

### 7.1 Export negative samples

In [ ]:
neg_data2 = pd.DataFrame()

neg_data2['drug_events'] = neg_data['drug_events'].apply(lambda x: ' '.join(str(i) for i in x))
neg_data2['procedure_codes'] = neg_data['procedure_codes'].apply(lambda x: ' '.join(x))

In [ ]:
neg_data2.head()

In [ ]:
# Split into train/validation; 200 validation samples and the rest are train
validation_neg = neg_data2.sample(n=200, random_state=3)
train_neg = neg_data2.drop(validation_neg.index)

In [ ]:
# Write as txt files into '../mimic_data/'
train_neg.to_csv(path_or_buf='../mimic_data/train_neg.txt', index=False, header=False, sep=' ', quoting = csv.QUOTE_NONE, escapechar = ' ')
validation_neg.to_csv(path_or_buf='../mimic_data/validation_neg.txt', index=False, header=False, sep=' ', quoting = csv.QUOTE_NONE, escapechar = ' ')

### 7.2 Export positive samples

In [ ]:
pos_data2 = pd.DataFrame()

pos_data2['drug_events'] = pos_data['drug_events'].apply(lambda x: ' '.join(str(i) for i in x))
pos_data2['procedure_codes'] = pos_data['procedure_codes'].apply(lambda x: ' '.join(x))

In [ ]:
pos_data2.head()

In [ ]:
# Split into train/validation; 200 validation samples and the rest are train
validation_pos = pos_data2.sample(n=200, random_state=3)
train_pos = pos_data2.drop(validation_pos.index)

In [ ]:
# Write as txt files into '../mimic_data/'
train_pos.to_csv(path_or_buf='../mimic_data/train_pos.txt', index=False, header=False, sep=' ', quoting = csv.QUOTE_NONE, escapechar = ' ')
validation_pos.to_csv(path_or_buf='../mimic_data/validation_pos.txt', index=False, header=False, sep=' ', quoting = csv.QUOTE_NONE, escapechar = ' ')